In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math
import os

## Load Data

In [2]:
import cifar10

Set the path for storing the data-set on your computer.

The CIFAR-10 data-set is about 163 MB and will be downloaded automatically if it is not located in the given path.

In [3]:
cifar10.maybe_download_and_extract()

Data has apparently already been downloaded and unpacked.


Load the class-names.

In [4]:
class_names = cifar10.load_class_names()
class_names

Loading data: data/CIFAR-10/cifar-10-batches-py/batches.meta


['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

Load the training-set. This returns the images, the class-numbers as integers, and the class-numbers as One-Hot encoded arrays called labels.

In [5]:
images_train, cls_train, labels_train = cifar10.load_training_data()

Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_1
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_2
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_3
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_4
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_5


Load the test-set.

In [6]:
images_test, cls_test, labels_test = cifar10.load_test_data()

Loading data: data/CIFAR-10/cifar-10-batches-py/test_batch


The CIFAR-10 data-set has now been loaded and consists of 60,000 images and associated labels (i.e. classifications of the images). The data-set is split into 2 mutually exclusive sub-sets, the training-set and the test-set.

In [7]:
print("Size of:")
print("- Training-set:\t\t{}".format(len(images_train)))
print("- Test-set:\t\t{}".format(len(images_test)))

Size of:
- Training-set:		50000
- Test-set:		10000


The data dimensions are used in several places in the source-code below. They have already been defined in the cifar10 module, so we just need to import them.

In [8]:
from cifar10 import img_size, num_channels, num_classes

The images are 32 x 32 pixels, but we will crop the images to 24 x 24 pixels.

In [9]:
img_size_cropped = 24

In [10]:
def pre_process_image(image, training):
    # This function takes a single image as input,
    # and a boolean whether to build the training or testing graph.
    
    if training:
        # For training, add the following to the TensorFlow graph.

        # Randomly crop the input image.
#        image = tf.random_crop(image, size=[img_size_cropped, img_size_cropped, num_channels])

        # Randomly flip the image horizontally.
        image = tf.image.random_flip_left_right(image)
        
        # Randomly adjust hue, contrast and saturation.
        image = tf.image.random_hue(image, max_delta=0.05)
        image = tf.image.random_contrast(image, lower=0.3, upper=1.0)
        image = tf.image.random_brightness(image, max_delta=0.2)
        image = tf.image.random_saturation(image, lower=0.0, upper=2.0)

        # Some of these functions may overflow and result in pixel
        # values beyond the [0, 1] range. It is unclear from the
        # documentation of TensorFlow whether this is
        # intended. A simple solution is to limit the range.

        # Limit the image pixels between [0, 1] in case of overflow.
        image = tf.minimum(image, 1.0)
        image = tf.maximum(image, 0.0)
    else:
        # For training, add the following to the TensorFlow graph.

        # Crop the input image around the centre so it is the same
        # size as images that are randomly cropped during training.
        image = tf.image.resize_image_with_crop_or_pad(image,
                                                       target_height=img_size_cropped,
                                                       target_width=img_size_cropped)

    return image

In [11]:
def pre_process(images, training):
    # Use TensorFlow to loop over all the input images and call
    # the function above which takes a single image as input.
    images = tf.map_fn(lambda image: pre_process_image(image, training), images)

    return images

## AlexNet

In [12]:
def weight_variable(shape, name, exp=0):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial + exp, name=name, trainable=True)

In [13]:
def conv2d(input_, out_channels, filter_height, filter_width, in_channels, strides):
    W = weight_variable(shape=[filter_height, filter_width, in_channels, out_channels], name="weight")
    b = weight_variable(shape=[out_channels], name="bias", exp=0.5)
    res = tf.nn.conv2d(input_, W, strides, padding='SAME')
    relu = tf.nn.relu(tf.nn.bias_add(res, b))
    return relu

In [14]:
def max_pool(input_, size, strides, name=None):
    return tf.nn.max_pool(input_, ksize=size,
                          strides=strides,
                          padding='SAME', name=name)

In [15]:
def fc(input_, input_size, output_size, keep_prob, name=None):
    fc_drop = tf.nn.dropout(input_, keep_prob, name="h_fc1_drop")

    W = weight_variable([input_size, output_size], "weight")
    b = weight_variable([output_size], "bias", exp=0.5)

    return tf.nn.relu(tf.nn.bias_add(tf.matmul(fc_drop, W), b, name='y_conv'))

In [16]:
def flatten(input_):
    flatten = tf.reshape(input_, shape=[-1, 2048])
    return flatten

In [17]:
def lrn(x, radius, alpha, beta, name, bias=1.0):
    return tf.nn.local_response_normalization(x, depth_radius = radius,
                                            alpha = alpha, beta = beta,
                                            bias = bias, name = name)

In [18]:
class AlexNet:
    def __init__(self, class_count, save_dir, save_path, batch_size):
        self.class_count = class_count
        self.save_dir = save_dir
        self.save_path = save_path
        self.batch_size = batch_size
        
    def _create_placeholders(self):
        with tf.name_scope("data"):
            self.ims = tf.placeholder(shape=[None, 32, 32, 3], dtype=tf.float32, name="x")
            self.y_true = tf.placeholder(shape=[None, self.class_count], dtype=tf.float32, name="y")
            self.global_step = tf.Variable(initial_value=0,
                          name='global_step', trainable=False)
        
    def _create_layers(self):
        #layer1: convolution 11x11 + RELU:
        with tf.name_scope("layer1"):
            layer1_1_conv = conv2d(self.x, 48, 11, 11, 3, [1, 4, 4, 1])
            layer1_2_conv = conv2d(self.x, 48, 11, 11, 3, [1, 4, 4, 1])
            norm1_1 = lrn(layer1_1_conv, 2, 2e-05, 0.75, name = 'norm1')
            norm1_2 = lrn(layer1_2_conv, 2, 2e-05, 0.75, name = 'norm1')
            layer1_1_pooling = max_pool(norm1_1, [1, 3, 3, 1], [1, 2, 2, 1])
            layer1_2_pooling = max_pool(norm1_2, [1, 3, 3, 1], [1, 2, 2, 1])
        
        with tf.name_scope("layer2"):
            layer2_1_conv = conv2d(layer1_1_pooling, 128, 5, 5, 48, [1, 1, 1, 1])
            layer2_2_conv = conv2d(layer1_2_pooling, 128, 5, 5, 48, [1, 1, 1, 1])
            norm1_1 = lrn(layer2_1_conv, 2, 2e-05, 0.75, name = 'norm1')
            norm1_2 = lrn(layer2_2_conv, 2, 2e-05, 0.75, name = 'norm1')            
            layer2_1_pooling = max_pool(norm1_1, [1, 3, 3, 1], [1, 2, 2, 1])
            layer2_2_pooling = max_pool(norm1_2, [1, 3, 3, 1], [1, 2, 2, 1])
        
        with tf.name_scope("layer3"):
            layer3_input = tf.nn.relu(tf.add(layer2_1_pooling, layer2_2_pooling))
            layer3_1_conv = conv2d(layer3_input, 192, 3, 3, 128, [1, 1, 1, 1])
            layer3_2_conv = conv2d(layer3_input, 192, 3, 3, 128, [1, 1, 1, 1])
            
        with tf.name_scope("layer4"):
            layer4_1_conv = conv2d(layer3_1_conv, 128, 3, 3, 192, [1, 1, 1, 1])
            layer4_2_conv = conv2d(layer3_2_conv, 128, 3, 3, 192, [1, 1, 1, 1])
            
        with tf.name_scope("layer5"):
            layer5_1_conv = conv2d(layer4_1_conv, 2048, 3, 3, 128, [1, 1, 1, 1])
            layer5_2_conv = conv2d(layer4_2_conv, 2048, 3, 3, 128, [1, 1, 1, 1])
            layer5_1_pooling = max_pool(layer5_1_conv, [1, 3, 3, 1], [1, 2, 2, 1])
            layer5_2_pooling = max_pool(layer5_2_conv, [1, 3, 3, 1], [1, 2, 2, 1])
        
        with tf.name_scope("layer6"):
            layer6_1_flatten = flatten(layer5_1_pooling)
            layer6_2_flatten = flatten(layer5_2_pooling)
            layer6_1_fc = fc(layer6_1_flatten, 2048, 2048, 1)
            layer6_2_fc = fc(layer6_2_flatten, 2048, 2048, 1)
            
        with tf.name_scope("layer7"):
            layer7_1_fc = fc(layer6_1_fc, 2048, 2048, 1)
            layer7_2_fc = fc(layer6_2_fc, 2048, 2048, 1)
            
        with tf.name_scope("layer8"):
            w_out = weight_variable([2048, self.class_count], "weight_out")
            b_out = weight_variable([self.class_count], "bias_out")
            self.logits = tf.add(tf.matmul(tf.nn.relu(layer7_1_fc + layer7_2_fc), w_out), b_out)        
    
    def _create_loss(self):
        with tf.name_scope("loss"):
            self.cross_entropy = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(labels=self.y_true, logits=self.logits))   
    
    def _create_optimizer(self):
        with tf.name_scope("optimizer"):
            self.train_step = tf.train.AdamOptimizer(1e-4).minimize(self.cross_entropy, global_step=self.global_step)
            
    def build_graph(self):
        self._create_placeholders()
        self.x = pre_process(self.ims, training=True)
        self._create_layers()
        self._create_loss()
        self._create_optimizer()
        self.saver = tf.train.Saver()
    
    def random_batch(self, images_train, labels_train):
        # Number of images in the training-set.
        num_images = len(images_train)

        # Create a random index.
        idx = np.random.choice(num_images,
                               size=self.batch_size,
                               replace=False)

        # Use the random index to select random images and labels.
        x_batch = images_train[idx, :, :, :]
        y_batch = labels_train[idx, :]
        return x_batch, y_batch
    
    def restore_checkpoints(self, session):
        try:
            print("Trying to restore last checkpoint ...")

            # Use TensorFlow to find the latest checkpoint - if any.
            last_chk_path = tf.train.latest_checkpoint(checkpoint_dir=self.save_dir)

            # Try and load the data in the checkpoint.
            self.saver.restore(session, save_path=last_chk_path)

            # If we get to this point, the checkpoint was successfully loaded.
            print("Restored checkpoint from:", last_chk_path)
        except:
#             If the above failed for some reason, simply
#             initialize all the variables for the TensorFlow graph.
            print("Failed to restore checkpoint. Initializing variables instead.")
            session.run(tf.global_variables_initializer())    
    
    def train_model(self, images_train, labels_train, num_iterations):
        y_pred = tf.nn.softmax(self.logits)
        y_pred_cls = tf.argmax(y_pred, axis=1)
        correct_prediction = tf.equal(y_pred_cls, tf.argmax(self.y_true, axis=1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True

        with tf.Session(config=config) as session:
            self.restore_checkpoints(session)

#             # Start-time used for printing time-usage below.        
            start_time = time.time()
        
            for i in range(num_iterations):
#                 # Get a batch of training examples.
#                 # x_batch now holds a batch of images and
#                 # y_true_batch are the true labels for those images.
                x_batch, y_true_batch = self.random_batch(images_train, labels_train)
                
#                 # Put the batch into a dict with the proper names
#                 # for placeholder variables in the TensorFlow graph.
                feed_dict_train = {self.ims: x_batch,
                                   self.y_true: y_true_batch}

#                 # Run the optimizer using this batch of training data.
#                 # TensorFlow assigns the variables in feed_dict_train
#                 # to the placeholder variables and then runs the optimizer.
#                 # We also want to retrieve the global_step counter.
                i_global, _ , entropy = session.run([self.global_step, self.train_step, self.cross_entropy],
                                          feed_dict=feed_dict_train)

#                 # Print status to screen every 100 iterations (and last).
                if (i_global % 100 == 0) or (i == num_iterations - 1):
#                     # Calculate the accuracy on the training-batch.
                    batch_acc, entropy = session.run([accuracy, self.cross_entropy],
                                            feed_dict=feed_dict_train)

#                     # Print status.
                    msg = "Global Step: {0:>6}, Training Batch Accuracy: {1:>6.1%}"
                    print(msg.format(i_global, batch_acc))
                    print(entropy)

                # Save a checkpoint to disk every 1000 iterations (and last).
                if (i_global % 1000 == 0) or (i == num_iterations - 1):
                    # Save all variables of the TensorFlow graph to a
                    # checkpoint. Append the global_step counter
                    # to the filename so we save the last several checkpoints.
                    self.saver.save(session,
                               save_path=self.save_path,
                               global_step=self.global_step)

                    print("Saved checkpoint.")
            
            self.print_test_accuracy(session)

            # Ending time.
            end_time = time.time()

            # Difference between start and end-times.
            time_dif = end_time - start_time

            # Print the time-usage.
            print("Time usage: " + str(timedelta(seconds=int(round(time_dif)))))
            
    def predict_cls(self, session, images, labels, cls_true):
        batch_size = 128

        # Number of images.
        num_images = len(images)

        # Allocate an array for the predicted classes which
        # will be calculated in batches and filled into this array.
        cls_pred = np.zeros(shape=num_images, dtype=np.int)

        # Now calculate the predicted classes for the batches.
        # We will just iterate through all the batches.
        # There might be a more clever and Pythonic way of doing this.

        # The starting index for the next batch is denoted i.
        i = 0

        while i < num_images:
            # The ending index for the next batch is denoted j.
            j = min(i + batch_size, num_images)

            # Create a feed-dict with the images and labels
            # between index i and j.
            
            self.batch_size = j - i + 1
            
            feed_dict = {self.x: images[i:j, :],
                         self.y_true: labels[i:j, :]}

            # Calculate the predicted class using TensorFlow.
            y_pred = tf.nn.softmax(alexNet.logits)
            y_pred_cls = tf.argmax(y_pred, axis=1)
            cls_pred[i:j] = session.run(y_pred_cls, feed_dict=feed_dict)

            # Set the start-index for the next batch to the
            # end-index of the current batch.
            i = j

        # Create a boolean array whether each image is correctly classified.
        correct = (cls_true == cls_pred)

        return correct, cls_pred

    def predict_cls_test(self, session):
        return self.predict_cls(session, images = images_test,
                           labels = labels_test,
                           cls_true = cls_test)
    
    def classification_accuracy(self, correct):
        # When averaging a boolean array, False means 0 and True means 1.
        # So we are calculating: number of True / len(correct) which is
        # the same as the classification accuracy.

        # Return the classification accuracy
        # and the number of correct classifications.
        return correct.mean(), correct.sum()
    
    def print_test_accuracy(self, session, show_example_errors=False,
                        show_confusion_matrix=False):

        # For all the images in the test-set,
        # calculate the predicted classes and whether they are correct.
        correct, cls_pred = self.predict_cls_test(session)

        # Classification accuracy and the number of correct classifications.
        acc, num_correct = self.classification_accuracy(correct)

        # Number of images being classified.
        num_images = len(correct)

        # Print the accuracy.
        msg = "Accuracy on Test-Set: {0:.1%} ({1} / {2})"
        print(msg.format(acc, num_correct, num_images))

        # Plot some examples of mis-classifications, if desired.
        if show_example_errors:
            print("Example errors:")
            plot_example_errors(cls_pred=cls_pred, correct=correct)

        # Plot the confusion matrix, if desired.
        if show_confusion_matrix:
            print("Confusion Matrix:")
            plot_confusion_matrix(cls_pred=cls_pred)



In [19]:
save_dir = 'checkpoints_alexNet/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
save_path = os.path.join(save_dir, 'cifar10_cnn')

alexNet = AlexNet(len(class_names), save_dir, save_path, 128)
alexNet.build_graph()

In [21]:
alexNet.train_model(images_train, labels_train, 98000)

Trying to restore last checkpoint ...
INFO:tensorflow:Restoring parameters from checkpoints_alexNet/cifar10_cnn-1000
Restored checkpoint from: checkpoints_alexNet/cifar10_cnn-1000
Global Step:   1100, Training Batch Accuracy:  29.4%
47.735
Global Step:   1200, Training Batch Accuracy:  29.4%
60.9607
Global Step:   1300, Training Batch Accuracy:  29.4%
39.3841
Global Step:   1400, Training Batch Accuracy:  35.3%
19.1757
Global Step:   1500, Training Batch Accuracy:  23.5%
20.5704
Global Step:   1600, Training Batch Accuracy:  29.4%
18.1247
Global Step:   1700, Training Batch Accuracy:  41.2%
16.4039
Global Step:   1800, Training Batch Accuracy:  29.4%
15.7346
Global Step:   1900, Training Batch Accuracy:  41.2%
9.91651
Global Step:   2000, Training Batch Accuracy:  29.4%
15.5065
Saved checkpoint.
Global Step:   2100, Training Batch Accuracy:  23.5%
8.51555
Global Step:   2200, Training Batch Accuracy:  35.3%
10.1586
Global Step:   2300, Training Batch Accuracy:  29.4%
12.225
Global Step

Global Step:  14000, Training Batch Accuracy:  17.6%
2.11696
Saved checkpoint.
Global Step:  14100, Training Batch Accuracy:  29.4%
1.85858
Global Step:  14200, Training Batch Accuracy:  23.5%
2.00325
Global Step:  14300, Training Batch Accuracy:  35.3%
2.06482
Global Step:  14400, Training Batch Accuracy:  41.2%
2.0965
Global Step:  14500, Training Batch Accuracy:  23.5%
1.75621
Global Step:  14600, Training Batch Accuracy:  35.3%
1.54923
Global Step:  14700, Training Batch Accuracy:  23.5%
1.84923
Global Step:  14800, Training Batch Accuracy:  29.4%
2.15459
Global Step:  14900, Training Batch Accuracy:  17.6%
1.97903
Global Step:  15000, Training Batch Accuracy:  35.3%
1.76519
Saved checkpoint.
Global Step:  15100, Training Batch Accuracy:  29.4%
1.61297
Global Step:  15200, Training Batch Accuracy:  17.6%
1.93001
Global Step:  15300, Training Batch Accuracy:  52.9%
1.59976
Global Step:  15400, Training Batch Accuracy:  11.8%
2.11472
Global Step:  15500, Training Batch Accuracy:  23.

Saved checkpoint.
Global Step:  27100, Training Batch Accuracy:  64.7%
1.24547
Global Step:  27200, Training Batch Accuracy:  29.4%
1.5544
Global Step:  27300, Training Batch Accuracy:  47.1%
1.43189
Global Step:  27400, Training Batch Accuracy:  35.3%
2.05217
Global Step:  27500, Training Batch Accuracy:  47.1%
1.6353
Global Step:  27600, Training Batch Accuracy:  29.4%
1.65828
Global Step:  27700, Training Batch Accuracy:  29.4%
1.93243
Global Step:  27800, Training Batch Accuracy:  41.2%
1.98951
Global Step:  27900, Training Batch Accuracy:  58.8%
0.984311
Global Step:  28000, Training Batch Accuracy:  41.2%
1.68987
Saved checkpoint.
Global Step:  28100, Training Batch Accuracy:  41.2%
1.71802
Global Step:  28200, Training Batch Accuracy:  47.1%
1.62669
Global Step:  28300, Training Batch Accuracy:  52.9%
1.3884
Global Step:  28400, Training Batch Accuracy:  35.3%
1.87165
Global Step:  28500, Training Batch Accuracy:  41.2%
1.34226
Global Step:  28600, Training Batch Accuracy:  29.4

Global Step:  40200, Training Batch Accuracy:  41.2%
1.57849
Global Step:  40300, Training Batch Accuracy:  23.5%
1.69464
Global Step:  40400, Training Batch Accuracy:  29.4%
1.93997
Global Step:  40500, Training Batch Accuracy:  58.8%
1.18186
Global Step:  40600, Training Batch Accuracy:  58.8%
1.48913
Global Step:  40700, Training Batch Accuracy:  29.4%
1.62071
Global Step:  40800, Training Batch Accuracy:  41.2%
1.69666
Global Step:  40900, Training Batch Accuracy:  58.8%
1.55368
Global Step:  41000, Training Batch Accuracy:  52.9%
1.32566
Saved checkpoint.
Global Step:  41100, Training Batch Accuracy:  41.2%
1.59799
Global Step:  41200, Training Batch Accuracy:  47.1%
1.33223
Global Step:  41300, Training Batch Accuracy:  52.9%
1.47584
Global Step:  41400, Training Batch Accuracy:  35.3%
1.66789
Global Step:  41500, Training Batch Accuracy:  58.8%
1.02276
Global Step:  41600, Training Batch Accuracy:  35.3%
1.54188
Global Step:  41700, Training Batch Accuracy:  47.1%
1.57467
Global

Global Step:  53300, Training Batch Accuracy:  29.4%
1.89114
Global Step:  53400, Training Batch Accuracy:  58.8%
1.03718
Global Step:  53500, Training Batch Accuracy:  35.3%
1.50655
Global Step:  53600, Training Batch Accuracy:  52.9%
1.54544
Global Step:  53700, Training Batch Accuracy:  35.3%
1.59096
Global Step:  53800, Training Batch Accuracy:  52.9%
1.13617
Global Step:  53900, Training Batch Accuracy:  47.1%
1.7302
Global Step:  54000, Training Batch Accuracy:  58.8%
1.19483
Saved checkpoint.
Global Step:  54100, Training Batch Accuracy:  52.9%
1.02363
Global Step:  54200, Training Batch Accuracy:  35.3%
1.36382
Global Step:  54300, Training Batch Accuracy:  64.7%
0.894267
Global Step:  54400, Training Batch Accuracy:  52.9%
1.60947
Global Step:  54500, Training Batch Accuracy:  47.1%
1.82183
Global Step:  54600, Training Batch Accuracy:  76.5%
0.897889
Global Step:  54700, Training Batch Accuracy:  70.6%
1.01201
Global Step:  54800, Training Batch Accuracy:  41.2%
1.62417
Globa

Global Step:  66400, Training Batch Accuracy:  64.7%
0.873202
Global Step:  66500, Training Batch Accuracy:  70.6%
1.01031
Global Step:  66600, Training Batch Accuracy:  52.9%
1.33534
Global Step:  66700, Training Batch Accuracy:  64.7%
0.757848
Global Step:  66800, Training Batch Accuracy:  70.6%
1.03723
Global Step:  66900, Training Batch Accuracy:  47.1%
1.26609
Global Step:  67000, Training Batch Accuracy:  70.6%
1.06632
Saved checkpoint.
Global Step:  67100, Training Batch Accuracy:  58.8%
1.06549
Global Step:  67200, Training Batch Accuracy:  58.8%
0.978537
Global Step:  67300, Training Batch Accuracy:  58.8%
1.04195
Global Step:  67400, Training Batch Accuracy:  47.1%
1.5468
Global Step:  67500, Training Batch Accuracy:  52.9%
1.06066
Global Step:  67600, Training Batch Accuracy:  58.8%
1.20989
Global Step:  67700, Training Batch Accuracy:  35.3%
1.31315
Global Step:  67800, Training Batch Accuracy:  76.5%
0.958314
Global Step:  67900, Training Batch Accuracy:  41.2%
1.40023
Glo

Global Step:  79500, Training Batch Accuracy:  58.8%
1.30513
Global Step:  79600, Training Batch Accuracy:  52.9%
1.08264
Global Step:  79700, Training Batch Accuracy:  41.2%
1.29745
Global Step:  79800, Training Batch Accuracy:  58.8%
1.08908
Global Step:  79900, Training Batch Accuracy:  58.8%
1.24632
Global Step:  80000, Training Batch Accuracy:  58.8%
1.17213
Saved checkpoint.
Global Step:  80100, Training Batch Accuracy:  70.6%
0.672836
Global Step:  80200, Training Batch Accuracy:  35.3%
1.31784
Global Step:  80300, Training Batch Accuracy:  64.7%
1.10749
Global Step:  80400, Training Batch Accuracy:  64.7%
1.32023
Global Step:  80500, Training Batch Accuracy:  47.1%
1.29003
Global Step:  80600, Training Batch Accuracy:  35.3%
1.17551
Global Step:  80700, Training Batch Accuracy:  58.8%
0.829566
Global Step:  80800, Training Batch Accuracy:  58.8%
1.23999
Global Step:  80900, Training Batch Accuracy:  58.8%
1.15458
Global Step:  81000, Training Batch Accuracy:  41.2%
1.53111
Save

Global Step:  92600, Training Batch Accuracy:  76.5%
0.950369
Global Step:  92700, Training Batch Accuracy:  52.9%
0.935379
Global Step:  92800, Training Batch Accuracy:  58.8%
1.02466
Global Step:  92900, Training Batch Accuracy:  52.9%
1.45396
Global Step:  93000, Training Batch Accuracy:  47.1%
1.81215
Saved checkpoint.
Global Step:  93100, Training Batch Accuracy:  70.6%
0.810621
Global Step:  93200, Training Batch Accuracy:  41.2%
1.42788
Global Step:  93300, Training Batch Accuracy:  58.8%
1.02743
Global Step:  93400, Training Batch Accuracy:  70.6%
0.809343
Global Step:  93500, Training Batch Accuracy:  64.7%
0.975552
Global Step:  93600, Training Batch Accuracy:  64.7%
0.839709
Global Step:  93700, Training Batch Accuracy:  52.9%
1.20857
Global Step:  93800, Training Batch Accuracy:  58.8%
1.1727
Global Step:  93900, Training Batch Accuracy:  47.1%
1.43285
Global Step:  94000, Training Batch Accuracy:  58.8%
1.32186
Saved checkpoint.
Global Step:  94100, Training Batch Accuracy